In [50]:
cd F:\MTP

F:\MTP


In [51]:
import os
import torch
from collections import defaultdict
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

In [52]:
path = os.listdir('EmoDB/wav')

We extract speaker embeddings(x-vectors) with a pretrained TDNN model using SpeechBrain from each utternace with a size of 512.

In [53]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

Each utterance from the database is name according to the scheme :
    Positions 1-2: number of speaker
    Positions 3-5: code for text
    Position 6: emotion (sorry, letter stands for german emotion word)
    Position 7: if there are more than two versions these are numbered a, b, c....

Example: 03a01Fa.wav is the audio file from Speaker 03 speaking text a01 with
the emotion "Freude" (Happiness in German).

So now,we extract the emotion from the filenames and consider it as y in our model.

In [54]:
emotion_code = {
    'W':0, #anger
    'L':1, #boredom
    'E':2, #disgust
    'A':3, #fear
    'F':4, #happy
    'T':5, #sad
    'N':6  #neutral
}

In [55]:
x_vec_size = 512
X = torch.zeros([len(path),x_vec_size])
y = torch.zeros([len(path),7],dtype = int)
for i in range(len(path)):
    signal, fs =torchaudio.load('EmoDB/wav/'+path[i])
    embeddings = classifier.encode_batch(signal)
    X[i] = embeddings[0][0]
    y[i][emotion_code[path[i][5]]]=1

In [56]:
print(X.shape,y.shape)

torch.Size([535, 512]) torch.Size([535, 7])


In [57]:
print(y[0])

tensor([0, 0, 0, 0, 1, 0, 0])


Convert them into numpy arrays so that we can train them easily

In [58]:
X=X.numpy()
y=y.numpy()

In [59]:
dict_X=defaultdict(list)
dict_y=defaultdict(list)
for i in range(len(path)):
    dict_X[path[i][:2]].append(X[i])
    dict_y[path[i][:2]].append(y[i])

To make this evaluation speaker independent, we separate the data speaker-wise and use leave-one-out validation.

In [60]:
dict = {0:"03",1:"08",2:"09",3:"10",4:"11",5:"12",6:"13",7:"14",8:"15",9:"16"}

In [61]:
#requirements


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from sklearn.metrics import accuracy_score

Since the x-vectors are extracted before the last layer from a pretrained model, we just add softmax layer to get the output layer.

In [67]:
def train_and_test(X,y,X_test,y_test):
    model = Sequential()
    model.add(Dense(7, input_shape=(len(X[0]),)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    model.fit(X,y,batch_size=20, epochs=200)
    predict_y = model.predict(X_test)
    predicted_test = predict_y
    np.where(np.argmax(predict_y[:]),1,0)
    for i in range(len(predict_y)):
        temp = np.argmax(predict_y[i])
        predict_y[i] = np.zeros((1,7))
        predict_y[i][temp] = 1
    acc=accuracy_score(y_test,predict_y)
    print(acc)
    return acc

In [68]:
temp = 9
acc = 0
accuracies = [0]*10
for i in range(10):
    a = list((dict_X[dict[i]] for i in range(10) if i != temp))
    b = list((dict_y[dict[i]] for i in range(10) if i != temp))
    X = np.concatenate(a)
    X_test = np.array(dict_X[dict[temp]])
    y_test = np.array(dict_y[dict[temp]])
    y = np.concatenate(b)
    temp -= 1
    print(X.shape,y.shape,X_test.shape,y_test.shape)
    temp_acc = train_and_test(X,y,X_test,y_test)
    accuracies[temp]=temp_acc
    print(f'Accuracy for iteration: {temp_acc}')
    acc += temp_acc 

(464, 512) (464, 7) (71, 512) (71, 7)
Epoch 1/200
24/24 [==============================] - 0s 2ms/step - loss: 10.9511 - accuracy: 0.1918
Epoch 2/200
24/24 [==============================] - 0s 1ms/step - loss: 4.3851 - accuracy: 0.3427
Epoch 3/200
24/24 [==============================] - 0s 2ms/step - loss: 2.9185 - accuracy: 0.4569
Epoch 4/200
24/24 [==============================] - 0s 1ms/step - loss: 2.5762 - accuracy: 0.5022
Epoch 5/200
24/24 [==============================] - 0s 1ms/step - loss: 1.8413 - accuracy: 0.6185
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 1.5320 - accuracy: 0.6638
Epoch 7/200
24/24 [==============================] - 0s 2ms/step - loss: 1.0508 - accuracy: 0.7392
Epoch 8/200
24/24 [==============================] - 0s 1ms/step - loss: 0.9925 - accuracy: 0.7478
Epoch 9/200
24/24 [==============================] - 0s 2ms/step - loss: 0.7261 - accuracy: 0.7866
Epoch 10/200
24/24 [==============================] - 0s 1ms/step - lo

24/24 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 83/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 84/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 85/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 86/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 87/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 88/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 89/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 90/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 91/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 92/200


24/24 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 164/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 165/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 166/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 167/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 168/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 169/200
24/24 [==============================] - ETA: 0s - loss: 0.0012 - accuracy: 1.00 - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 170/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 171/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 172/200
24/24 [==============================] - 0s 2ms/s

24/24 [==============================] - 0s 1ms/step - loss: 0.0185 - accuracy: 1.0000
Epoch 36/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0185 - accuracy: 1.0000
Epoch 37/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0186 - accuracy: 0.9979
Epoch 38/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0146 - accuracy: 1.0000
Epoch 39/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0167 - accuracy: 1.0000
Epoch 40/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 41/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0133 - accuracy: 1.0000
Epoch 42/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 43/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0144 - accuracy: 1.0000
Epoch 44/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0113 - accuracy: 1.0000
Epoch 45/200


24/24 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 117/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 118/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 119/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 120/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 121/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 122/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 123/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 124/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 125/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0022 - accuracy: 1.0000
Epoc

...

24/24 [==============================] - 0s 1ms/step - loss: 0.0197 - accuracy: 0.9979
Epoch 35/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 36/200
24/24 [==============================] - ETA: 0s - loss: 0.0314 - accuracy: 1.00 - 0s 1ms/step - loss: 0.0173 - accuracy: 1.0000
Epoch 37/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0154 - accuracy: 1.0000
Epoch 38/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0188 - accuracy: 0.9979
Epoch 39/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0205 - accuracy: 0.9958
Epoch 40/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0172 - accuracy: 0.9979
Epoch 41/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0135 - accuracy: 1.0000
Epoch 42/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0197 - accuracy: 0.9958
Epoch 43/200
24/24 [==============================] - 0s 2ms/step - los

24/24 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 116/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 117/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 118/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 119/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 120/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 121/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 122/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 123/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 124/200
24/24 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 1.0000
Epoc

24/24 [==============================] - 0s 1ms/step - loss: 7.4699e-04 - accuracy: 1.0000
Epoch 196/200
24/24 [==============================] - 0s 1ms/step - loss: 7.2799e-04 - accuracy: 1.0000
Epoch 197/200
24/24 [==============================] - 0s 1ms/step - loss: 7.2771e-04 - accuracy: 1.0000
Epoch 198/200
24/24 [==============================] - 0s 1ms/step - loss: 7.1621e-04 - accuracy: 1.0000
Epoch 199/200
24/24 [==============================] - 0s 2ms/step - loss: 6.9794e-04 - accuracy: 1.0000
Epoch 200/200
24/24 [==============================] - 0s 2ms/step - loss: 6.9957e-04 - accuracy: 1.0000
0.5862068965517241
Accuracy for iteration: 0.5862068965517241
(486, 512) (486, 7) (49, 512) (49, 7)
Epoch 1/200
25/25 [==============================] - 0s 2ms/step - loss: 8.9690 - accuracy: 0.2305
Epoch 2/200
25/25 [==============================] - 0s 2ms/step - loss: 3.9441 - accuracy: 0.3765
Epoch 3/200
25/25 [==============================] - 0s 1ms/step - loss: 2.4699 - accu

25/25 [==============================] - 0s 1ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 68/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 69/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 70/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 71/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 72/200
25/25 [==============================] - ETA: 0s - loss: 0.0063 - accuracy: 1.00 - 0s 1ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 73/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0063 - accuracy: 1.0000
Epoch 74/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 75/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 76/200
25/25 [==============================] - 0s 1ms/step - los

25/25 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 148/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 149/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 150/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 151/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 152/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 153/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 154/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 155/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 156/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoc

leave-one-speaker-out validation accuracies

In [69]:
accuracies

[0.5862068965517241,
 0.6976744186046512,
 0.7631578947368421,
 0.6363636363636364,
 0.7428571428571429,
 0.7704918032786885,
 0.8405797101449275,
 0.7142857142857143,
 0.9154929577464789,
 0.8979591836734694]

In [70]:
print("Overall accuracy :"+ str(acc*10)+"%" )

Overall accuracy :75.65069358243277%


In [66]:
M31_X,M31_y=np.array(dict_X["03"]),np.array(dict_y["03"])
F34_X,F34_y=np.array(dict_X["08"]),np.array(dict_y["08"])
F21_X,F21_y=np.array(dict_X["09"]),np.array(dict_y["09"])
M32_X,M32_y=np.array(dict_X["10"]),np.array(dict_y["10"])
M26_X,M26_y=np.array(dict_X["11"]),np.array(dict_y["11"])
M30_X,M30_y=np.array(dict_X["12"]),np.array(dict_y["12"])
F32_X,F32_y=np.array(dict_X["13"]),np.array(dict_y["13"])
F35_X,F35_y=np.array(dict_X["14"]),np.array(dict_y["14"])
M25_X,M25_y=np.array(dict_X["15"]),np.array(dict_y["15"])
F31_X,F31_y=np.array(dict_X["16"]),np.array(dict_y["16"])

In [47]:
temp_acc

0.7551020408163265